In [1]:
import getpass
import os
import pprint
from dotenv import load_dotenv

load_dotenv()

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

tracing_v2 = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

We can simply pass a message to the model using `.invoke` method because all `ChatModel`s are "Runnables".

In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-273e2d59-3be4-40a3-adfa-fb822fb92388-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

The model on it's own has no concep of state, if you ask a followup question:

In [3]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have the ability to know your name.", response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 12, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-94db4df3-d4bc-409b-acaa-8e6ef1cd3137-0', usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27})

It will have no context from what was previously asked. To get around this, we need to pass the conversation history into the model.

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. How can I assist you today, Bob?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 35, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bc975790-f777-4cb9-9cfb-729913a3a080-0', usage_metadata={'input_tokens': 35, 'output_tokens': 14, 'total_tokens': 49})

# Message History
The Message History class can wrap the model ad make it stateful. It can keep track of inputs and outputs and store them in some datastore. These can them be loaded into the messages array and pass them into the chain as part of the input.

You will need to install `langchain-community`, an integration in there will be used to store message history.

In [5]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

To simulate a chat app, we will need to pass in information that may not be part of the input directly. This can be stored in a `config` variable. In this case, we want to include a `session_id`.

In [6]:
config = {"configurable": {"session_id": "abc2"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello Bob! How are you today?'

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob.'

Our chat history is now being stored against the `session_id` abc2. If we change the config to reference a different `session_id`, the conversation will start fresh, but you can always go back to the original conversation since we are persisting it in a database.

In [9]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I do not know your name."

In [10]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob.'

# Prompt Templates
We can use prompt templates to add a system message with custom instructions alongside the user message. 

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [12]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello, Bob! How can I assist you today?'

We can now wrap this in the same Messages History object as before.

In [13]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [14]:
config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello Jim! How can I assist you today?'

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Let's make our prompt a little more complicated and get the assistant to response in a different language.

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

We can now wrap this in the Message History class. This time, there are multiple keys in the input (`messages` and `language`), so we need to specify the correct key to use to save the change history.

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [18]:
config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [19]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es Todd. ¿Hay algo más en lo que pueda ayudarte?'

# Managing Conversation History
It is important to understand how to manage conversation history, if left unmanaged, the list of messages will grow and potentially overflow the context window of the LLM. It is important to add a step that limits the size of the messages that are passed in.

Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.

We can do this by adding a simple step in front of the prompt that modifies the messages key appropriately, and then wrap that new chain in the Message History class.

LangChain comes with a few built-in helpers for managing a list of messages. In this case we'll use the trim_messages helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages:

In [20]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt.



In [21]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) # 1. Trim message history
    | prompt # 2. User prompt template to create message array.
    | model # 3. Send this to the model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, but I don't have access to your personal information. How can I assist you today?"

The model no longer has access to the message about our name. but if we ask about information that is within the last few messages, it remembers:

In [22]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what is 2 + 2.'

We can now wrap this in message history.

In [23]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have access to your personal information. How can I assist you today?"

As expected, the first message where we stated our name has been trimmed. Plus there's now two new messages in the chat history (our latest question and the latest response). This means that even more information that used to be accessible in our conversation history is no longer available! In this case our initial math question has been trimmed from the history as well, so the model no longer knows about it:

In [24]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked any math problems yet. Feel free to ask any math-related question you have, and I'll be happy to help you solve it."

You can see exactly what is happening, and which messages from chat history is sent to the model in [langchain_tutorial_python - LangSmith](https://smith.langchain.com/o/54da7528-d21e-5138-86a9-50d5d525d26f/projects/p/516b66e3-af18-4bb8-81b9-b3be8c3606a3?timeModel=%7B%22duration%22:%227d%22%7D&peek=a01e3883-1985-4bc8-b759-c2b8356eff06).


# Streaming
Super important UX consideration for chatbot applications is streaming. All chains expose a `.stream` method, we can use this method to get back a streaming response from our message history runnable.

In [25]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


|Hi| Todd|!| Sure|,| here|'s| a| joke| for| you|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|Because| it| was| two| tired|!||